In [ ]:
# Colab setup
# Enable GPU before running
# If you have a GPU available, ensure the runtime is configured to use it before
# executing heavy cells. In local Windows PowerShell you may need to install CUDA
# and the appropriate GPU drivers; in cloud notebooks select a GPU runtime.
# You can check availability in Python with:
# import torch; print(torch.cuda.is_available(), torch.cuda.device_count())

In [ ]:
# Repository and dependency setup
# This cell clones the repository and installs Python dependencies used by the notebooks.
# Note: the `!` prefix runs shell commands in notebook environments (Colab/Jupyter).
# On Windows PowerShell you may run the equivalent commands directly in a terminal instead
# of using the `!` magic. Review the commands before executing in your environment.
# clone the ADIS repository
!git clone https://github.com/sathishkumar67/ADIS.git
# move the files to the current directory (path used for hosted runtimes)
!mv /content/ADIS/* /content/ # change this path as needed to move files to the current directory
# upgrade pip to the latest version
!pip install --upgrade pip
# install the required packages listed in requirements.txt
!pip install -r requirements.txt

In [ ]:
# Necessary imports
# Import only lightweight modules here so the cell runs quickly when inspecting the notebook.
# Heavy imports (e.g., torch) are deferred to model code to avoid long import times when
# simply reading the notebook.
from __future__ import annotations
import os
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model
# `YOLO11Model` is the project's model wrapper; it provides convenience methods for loading
# checkpoints and running evaluation/training. See `model.py` for implementation details.

In [ ]:
# Local and dataset configuration
# Set LOCAL_DIR to the working directory where datasets and models will be stored/loaded.
LOCAL_DIR = os.getcwd()

# HuggingFace repository identifiers used for dataset and model downloads
REPO_ID = "pt-sk/ADIS"
DATASET_NAME = "balanced_dataset"
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
REPO_TYPE = "dataset"
MODEL_REPO_TYPE = "model"

# Data paths: these are assembled from LOCAL_DIR and DATASET_NAME. Ensure the dataset is
# extracted into the expected layout (train/val/test).
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"

# Dataset metadata
NUM_CLASSES = 10
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake']
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
# The data yaml file written below is used by the model validation/training routines.

In [ ]:
# Download and extract the dataset
# This cell downloads the dataset archive from the HuggingFace hub and extracts it into LOCAL_DIR.
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
# `unzip_file` is a small helper in utils.py that extracts the archive contents into LOCAL_DIR.
unzip_file(DATASET_PATH, LOCAL_DIR)

# Remove the downloaded archive to save disk space. If the file is already deleted this will error;
# ensure you are comfortable with the behavior or guard with `os.path.exists` in interactive runs.
os.remove(DATASET_PATH)

In [ ]:
# Create a data.yaml file used by training/validation routines
# The YAML lists train/val/test folders and the class names in the format expected by
# the project's validation and training utilities.
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file (this will overwrite an existing file at the same path)
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    # The file is now available for model.val(...) and training entrypoints.

## Define which model do you want to evaluate with

### Unoptimized Models:
  1. yolov11n_without_bohb_tuned_ckpt.pt
  2. yolov11s_without_bohb_tuned_ckpt.pt
  3. yolov11m_without_bohb_tuned_ckpt.pt

### Optimized Models:
  1. yolov11n_bohb_tuned_ckpt.pt
  2. yolov11s_bohb_tuned_ckpt.pt
  3. yolov11m_bohb_tuned_ckpt.pt

Enter the model name to download the saved model from huggingface hub


In [ ]:
# Select and download a model checkpoint from HuggingFace hub
# Choose one of the available checkpoints listed in the markdown cell above. Do not hardcode
# a model variant here if you prefer to set it externally; this variable controls which file
# the notebook attempts to download.
MODEL_NAME = "yolov11n_bohb_tuned_ckpt.pt"

# download the model checkpoint into LOCAL_DIR
hf_hub_download(repo_id=REPO_ID, filename=MODEL_NAME, repo_type=MODEL_REPO_TYPE, local_dir=LOCAL_DIR)

# load the model with the pretrained weights. The YOLO11Model wrapper provides convenience
# methods for evaluation and training; see `model.py` for details.
model = YOLO11Model(MODEL_NAME, verbose=True)

## Model Evaluation
To evaluate the model specify the split that you want to evaluate on. The options are:
  1. train
  2. val
  3. test


In [ ]:
# Evaluate the model on a chosen dataset split
# Set SPLIT to one of: 'train', 'val', or 'test' depending on which subset you want to evaluate.
SPLIT = "val"
import time
start_time = time.time()
# Run the model's validation routine; it expects a data YAML path and a split name.
model.val(data=DATA_YAML_FILE, split=SPLIT)
print(f"Time taken to evaluate on {SPLIT} split: {time.time() - start_time} seconds")

## Model Inference

In [ ]:
# Inference on a single image
# Specify the path to an image file for running inference. If left blank, no inference
image_path = "" 
# Run inference on a single image
if image_path:
    results = model(image_path)
    # display the results
    results.show()